In [ ]:
import pandas as pd
import requests
import zipfile
import os

pd.options.display.float_format = '{:.4f}'.format

In [ ]:
ano = "2022"
mes = "12"
url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano}{mes}.zip'

In [ ]:
download = requests.get(url)

with open(f"inf_diario_fi_{ano}{mes}.zip", "wb") as arquivo_cvm:

    arquivo_cvm.write(download.content) 
    
arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano}{mes}.zip")

    

In [ ]:
dados_fundos = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep = ";", encoding = 'ISO-8859-1')

dados_fundos

In [ ]:
dados_cadastro = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', 
                             sep = ";", encoding = 'ISO-8859-1')

dados_cadastro = dados_cadastro[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

dados_cadastro = dados_cadastro.drop_duplicates()

dados_cadastro

In [ ]:
data_inicio_mes = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[-0]
data_fim_mes = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[-1]
        

dados_fundos_filtrado = dados_fundos[(dados_fundos['DT_COMPTC'].isin([data_inicio_mes, data_fim_mes]))]
                           
dados_fundos_filtrado

In [ ]:
base_final = pd.merge(dados_fundos_filtrado, dados_cadastro, how = "left", 
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])

base_final = base_final[['CNPJ_FUNDO', 'DENOM_SOCIAL', 
                         'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]

base_final = base_final.dropna()

base_final

In [ ]:
base_final[base_final['DENOM_SOCIAL'].str.contains("ARCA GRÃO")]